[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%2010/Worksheet%2010-2%20Finite%20Difference%20Methods%20for%20BVPs.ipynb)

# Worksheet 10-2: Finite Difference Method for Beam Deflection BVPs

## ENGR 240 - Week 10: Boundary Value Problems

**Learning Goals:** Apply finite differences to solve BVPs, formulate matrices by hand, understand boundary condition effects

In this worksheet, we'll solve a classic structural engineering problem using the finite difference method. You'll see how to transform a differential equation into a linear system that can be solved with standard numerical methods.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 6)

# 1. Problem Setup (10 min)

**Simply supported beam with settlement:**
- Length L = 4 m, uniform load q = 1000 N/m
- Left support: y(0) = 0, Right support: y(L) = δ = 0.005 m (settlement)
- Governing ODE: d²y/dx² = q/(EI) = k (constant)

**Analytical solution:** y(x) = (q/24EI)x(L³-2L²x+x³) + δx/L

In [ ]:
# Parameters
L, q, E, I, delta = 4.0, 1000.0, 200e9, 8.33e-6, 0.005
k = q / (E * I)
print(f"Constant k = q/(EI) = {k:.2e} m⁻²")

# Analytical solution
def y_analytical(x):
    return (q/(24*E*I)) * x * (L**3 - 2*L**2*x + x**3) + (delta * x / L)

# Plot reference solution
x_plot = np.linspace(0, L, 100)
y_plot = y_analytical(x_plot)
plt.plot(x_plot, y_plot*1000, 'b-', linewidth=2, label='Analytical')
plt.xlabel('x (m)'); plt.ylabel('y (mm)'); plt.grid(True); plt.legend(); plt.show()
print(f"Max deflection: {min(y_plot)*1000:.2f} mm")
print("\nNotice the beam deflects downward due to the load, but the right end")
print("is forced upward due to the 5mm settlement of the support.")

# Task 2: Recall Finite Differences (8 min)

Before diving into the problem, let's recall the fundamental finite difference approximation you learned in previous weeks.

**🤔 Complete this:** d²y/dx²|ᵢ ≈ (yᵢ₊₁ - _____ + yᵢ₋₁)/h²

**Answer:** 2yᵢ

This second-order central difference formula is the key to transforming our differential equation into algebraic equations.

**6-node grid:** h = L/5 = 0.8 m
```
Node:  0    1    2    3    4    5
x:     0   0.8  1.6  2.4  3.2   4
BC:   y=0   ?    ?    ?    ?   y=δ
```
Interior nodes 1,2,3,4 → 4 equations needed

We have 4 unknown deflections at the interior nodes, so we need exactly 4 equations to solve the system.

In [ ]:
# Grid setup
n_nodes = 6
h = L / (n_nodes - 1)
x_nodes = np.linspace(0, L, n_nodes)
print(f"Grid spacing h = {h:.1f} m")
print("Nodes:", [f"x_{i}={x:.1f}" for i, x in enumerate(x_nodes)])

# 3. Hand Formulation (20 min)

**Step 1:** Apply FD at each interior node: (yᵢ₋₁ - 2yᵢ + yᵢ₊₁)/h² = k

Rearrange: yᵢ₋₁ - 2yᵢ + yᵢ₊₁ = kh²

**Step 2:** Write equations for nodes 1,2,3,4:
- Node 1: y₀ - 2y₁ + y₂ = kh² → -2y₁ + y₂ = kh² (since y₀=0)
- Node 2: y₁ - 2y₂ + y₃ = kh²
- Node 3: y₂ - 2y₃ + y₄ = kh²
- Node 4: y₃ - 2y₄ + y₅ = kh² → y₃ - 2y₄ = kh² - δ (since y₅=δ)

**✏️ YOUR TURN:** Complete the matrix equation Ay = b:

$$\begin{bmatrix}-2&1&0&0\\?&?&?&?\\?&?&?&?\\?&?&?&?\end{bmatrix}\begin{bmatrix}y_1\\y_2\\y_3\\y_4\end{bmatrix}=\begin{bmatrix}kh^2\\?\\?\\?\end{bmatrix}$$

In [ ]:
# Calculate RHS constant
rhs = k * h**2
print(f"kh² = {rhs:.6f}")

# SOLUTION: Fill in your hand-calculated matrix
A = np.array([
    [-2.0,  1.0,  0.0,  0.0],  # Node 1
    [ 1.0, -2.0,  1.0,  0.0],  # Node 2
    [ 0.0,  1.0, -2.0,  1.0],  # Node 3
    [ 0.0,  0.0,  1.0, -2.0]   # Node 4
])

b = np.array([
    rhs,        # Node 1: kh²
    rhs,        # Node 2: kh²
    rhs,        # Node 3: kh²
    rhs - delta # Node 4: kh² - δ  ← BOUNDARY CONDITION EFFECT!
])

print("Matrix A:")
print(A)
print(f"\nVector b: {b}")
print(f"\nKey insight: δ = {delta} affects b[3], not matrix A!")
print("This is a fundamental principle: boundary conditions modify the")
print("right-hand side vector, while the coefficient matrix A depends")
print("only on the interior finite difference stencil.")

# Task 4: Solve & Compare (12 min)

Now let's solve our linear system and see how well our finite difference approximation matches the analytical solution. This is the moment of truth!

In [ ]:
# Solve linear system
y_interior = np.linalg.solve(A, b)
print("Interior node solutions:")
for i, y in enumerate(y_interior, 1):
    print(f"y_{i} = {y:.6f} m = {y*1000:.3f} mm")

# Complete solution (add boundary nodes)
y_fd = np.zeros(n_nodes)
y_fd[0] = 0.0
y_fd[1:5] = y_interior
y_fd[5] = delta

# Analytical at nodes
y_exact_nodes = y_analytical(x_nodes)

# Plot comparison
plt.plot(x_plot, y_plot*1000, 'b-', linewidth=2, label='Analytical')
plt.plot(x_nodes, y_fd*1000, 'ro-', markersize=8, label='6-node FD')
plt.plot([0, L], [0, delta*1000], 'gs', markersize=10, label='Boundary Conditions')
plt.xlabel('x (m)'); plt.ylabel('y (mm)'); plt.grid(True); plt.legend(); plt.show()

# Error analysis
errors = np.abs(y_fd - y_exact_nodes) * 1000
print(f"\nMax error: {np.max(errors):.4f} mm")
print(f"RMS error: {np.sqrt(np.mean(errors**2)):.4f} mm")

# Task 5: Mesh Refinement (10 min)

One of the beautiful aspects of numerical methods is that we can systematically improve accuracy by refining our discretization. Let's see how the solution converges as we use more nodes.

In [ ]:
def solve_beam_fd(n_nodes):
    """Solve beam BVP with n_nodes using finite differences"""
    h = L / (n_nodes - 1)
    x = np.linspace(0, L, n_nodes)
    n_int = n_nodes - 2
    
    # Build tridiagonal matrix
    A = np.diag(-2*np.ones(n_int)) + np.diag(np.ones(n_int-1), 1) + np.diag(np.ones(n_int-1), -1)
    
    # RHS vector
    b = np.full(n_int, k * h**2)
    b[-1] -= delta  # BC effect
    
    # Solve and construct full solution
    y_int = np.linalg.solve(A, b)
    y_full = np.zeros(n_nodes)
    y_full[1:-1] = y_int
    y_full[-1] = delta
    
    # Calculate error
    y_exact = y_analytical(x)
    max_error = np.max(np.abs(y_full - y_exact)) * 1000
    
    return x, y_full, max_error

# Convergence study
node_counts = [6, 11, 21, 41]
errors = []
h_values = []

plt.figure(figsize=(12, 5))
plt.subplot(1,2,1)
plt.plot(x_plot, y_plot*1000, 'k-', linewidth=3, label='Analytical')

colors = ['red', 'green', 'orange', 'purple']
for i, (n, color) in enumerate(zip(node_counts, colors)):
    x, y, err = solve_beam_fd(n)
    h = L / (n - 1)
    h_values.append(h)
    errors.append(err)
    plt.plot(x, y*1000, 'o-', color=color, label=f'{n} nodes', markersize=4)

plt.xlabel('x (m)'); plt.ylabel('y (mm)'); plt.title('Convergence with Mesh Refinement')
plt.grid(True); plt.legend()

plt.subplot(1,2,2)
plt.loglog(h_values, errors, 'bo-', linewidth=2, markersize=8)
plt.loglog(h_values, errors[0]*(np.array(h_values)/h_values[0])**2, 'r--', label='O(h²)')
plt.xlabel('Grid spacing h (m)'); plt.ylabel('Max error (mm)')
plt.title('Error vs Grid Spacing'); plt.grid(True); plt.legend()
plt.tight_layout(); plt.show()

print("Convergence Results:")
for n, h, err in zip(node_counts, h_values, errors):
    print(f"{n:2d} nodes: h={h:.3f}m, error={err:.4f}mm")
print(f"\nError reduction factor: {errors[0]/errors[-1]:.1f}x")

# Task 6: Key Takeaways (5 min)

Congratulations! You've successfully implemented the finite difference method for a boundary value problem. Let's summarize what you've learned and where this knowledge can take you.

**✅ What we learned:**
- FD method: substitute approximations into ODE
- Each interior node → one equation
- Non-zero BCs affect RHS vector, not coefficient matrix
- Finer meshes → better accuracy (O(h²) convergence)

**🔧 Engineering relevance:**
- Foundation settlement analysis
- Variable loading: q(x)
- Different BCs: cantilever, fixed-fixed
- Extension to 2D/3D problems

**🎯 The big picture:** This method scales up to solve problems with thousands of unknowns, making it practical for real engineering analysis. Modern structural analysis software uses these same principles!

**📈 Next:** Finite element method for complex geometries!